In [1]:
## Preamble: Package Loading
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import itertools as iter
import math

<h4> Block Diagonal Matrix Function </h4>

In [2]:
def blkdiag(mat,nb):
    """
INPUTS
mat     Square Matrix which will form the block in a block diagonal matrix
nb      Number of diagonal block in output matrix 

OUTPUT
v       Block diagonal matrix of dimension ( nb*mat.shape[0] x nb*mat.shape[0] )
    """
    # Initializing the varcov matrix for all crosssections
    v = np.hstack((mat,np.zeros((mat.shape[0],(nb-1)*(mat.shape[1])))))
    # Registry matrix used in following loops
    vreg = np.eye(nb-1)
    for j in np.arange(nb-1):
        # Initializing current block of rows 
        pv = np.zeros((mat.shape[0],mat.shape[0]))
        # Horiz Stacking either zeros or var_err depending on ve_reg[j,i]
        for i in np.arange(nb-1):
            if vreg[j,i] == 1: # Stack var_err onto pv_err
                pv = np.hstack((pv,mat))
            if vreg[j,i] == 0: # Stack zeros onto pv_err
                pv = np.hstack((pv,np.zeros((mat.shape[0],mat.shape[0]))))
        # Vertically stacking block rows on top of one another
        v = np.vstack((v,pv))
    return v

#### Block Diagonal Matrix Function Demonstration

In [3]:
blkdiag(np.ones((3,3)),4)

array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.]])

<h3> DGP Inputs </h3>

In [4]:
# Setting a Seed
np.random.seed([10])

# Number of Time Periods
ntp = 10
# Number of Cross-Sections
ncs = 3
# Number of Endogenous Variables in Primary Equation
n_end = 2
# Number of Endogenous Variables in Primary Equation
n_exo = 2
# Total Number of Instruments
t_inst = 10
# Number of Instruments per Crossection
c_inst = 3
# Indicator for whether to force additive non linear cntrl function. 1 = yes 
frc = 0

# Vector of exog off diagonal covariances i.e. cov(Z2t_l,Z2t_(l+j)) = ex_vpro[j-1]
ex_vpro = [0.5]
# Vector of inst off diagonal covariances i.e. cov(Wt_l,Wt_(l+j)) = inst_vpro[j-1]
inst_vpro = [0.5 , 0.25 ]
# Vector of error off diagonal covariances i.e. cov(Wt_l,Wt_(l+j)) = inst_vpro[j-1]
err_vpro = [0.8 , 0.36 ]

<h2> DGP Generation Code </h2>

In [86]:
# Collection of all Coefficients on Instruments 
icoeffs_reg = list(iter.product([-1,1],repeat = c_inst))
# Collection of all Coefficients on Exogenous Variables
excoeffs_reg = list(iter.product([-1,1],repeat = n_exo))
# Registry of instrument assignments
insts_reg = list(iter.combinations(np.arange(1,t_inst+1),c_inst))

# List of which instruments (col #'s) are used for each crossection 
icr = [insts_reg[np.random.randint(len(insts_reg))] for i in range(ncs)]

# All permutation of combinations of n_end icoefficient vectors  
picfs = list(iter.permutations(range(0,len(icoeffs_reg)),n_end))
icfs = [ picfs[i] for i in np.random.randint(len(picfs),size = ncs)]

# All permutation of combinations of n_end icoefficient vectors  
pxcfs = list(iter.permutations(range(0,len(excoeffs_reg)),n_end))
xcfs = [ pxcfs[i] for i in np.random.randint(len(pxcfs),size = ncs)]
            
# Initializing Coefficient Matrix            
coeff = np.zeros((n_end,ncs,t_inst+n_exo))
for j in range(n_end):
    for i in range(ncs):
        for k in range(n_exo):
            coeff[j,i,k] = excoeffs_reg[xcfs[i][j]][k]
        
        for k in range(c_inst): 
            coeff[j,i,icr[i][k]+n_exo-1] = icoeffs_reg[icfs[i][j]][k]            
            
## Joint Distribution of Exogenous regressors
# Vector of Means (=0)
mu_ex = np.zeros(n_exo)
# Diagonal matrix of variaces (=1)
var_ex = np.eye(n_exo)
# Variance Covariance Matrix Generation or EACH crossection
for i in np.arange(len(ex_vpro)):
    var_ex = (var_ex + ex_vpro[i]*np.eye(n_exo,k=i+1)
                         + ex_vpro[i]*np.eye(n_exo,k=-(i+1))) 
# Exogenous regressor mean vector for ALL Crossections
Mu_ex = np.tile(mu_ex,ncs) 
# Exogenous regressor variance covariance matrix for ALL Crossections
V_ex = blkdiag(var_ex,ncs)
    
## Joint Distribution of Instruments for all cross sections
# Vector of Means (=0)
mu_inst = np.zeros(t_inst)
# Diagonal Matrix of Variaces (=1)
var_inst = np.eye(t_inst)
# Variance Covariance Matrix Generation
for i in np.arange(len(inst_vpro)):
    var_inst = (var_inst + inst_vpro[i]*np.eye(t_inst,k=i+1) 
                         + inst_vpro[i]*np.eye(t_inst,k=-(i+1)))
    
## Joint Distribution of Error Terms for EACH crossection
# Vector of means
mu_err = np.zeros(n_end+1)
# Diagonal Matrix of Variances 
var_err = np.eye(n_end+1)
# Variance Covariance Matrix Generation
if frc == 0 : 
    # Var Cov matrix for correlated errors ==> additive linear control functions 
    for i in np.arange(len(err_vpro)):
        var_err = (var_err + err_vpro[i]*np.eye(n_end+1,k=i+1) 
                             + err_vpro[i]*np.eye(n_end+1,k=-(i+1)))
# Error term mean vector for ALL Crossections
Mu_err = np.tile(mu_err,ncs)        
# Error term variance covariance matrix for ALL Crossections
V_err = blkdiag(var_err,ncs)

## Variable Generation
# Exogenous Regressor Generation
Ex = np.random.multivariate_normal(Mu_ex,V_ex,ntp)
# Instruments Generation
Inst = np.random.multivariate_normal(mu_inst,var_inst,ntp)
# Error Terms Generation 
Err = np.random.multivariate_normal(Mu_err,V_err,ntp)

## Variable Name Generation
ex_nms = [''.join(['Z2',str(i),',',str(j)]) for i in list(range(1,ncs+1)) for j in list(range(1,n_exo+1))]

# Constructing names for instruments
inst_nms = [''.join(['W',str(i)]) for i in list(range(1,t_inst+1))]

# Constructing names for error terms
err_nm1 = ['e' if val == n_end+1 else 'V' for val in  list(range(1,n_end+2))*ncs]
err_nm2 = [ str(i) for y in range(1,n_end+2) for i in iter.repeat(y,n_end+1)]
err_nm3 = ['' if val == n_end+1 else ''.join([',',str(val)]) for val in list(range(1,n_end+2))*ncs]
err_nm  = [''.join([err_nm1[i],err_nm2[i],err_nm3[i]]) for i in range(len(err_nm1))]

## Optional Data Frame Generation 
Ex_df = pd.DataFrame(Ex,columns = ex_nms)
Inst_df = pd.DataFrame(Inst, columns = inst_nms)
Err_df = pd.DataFrame(Err,columns = err_nm)

## Generating Endogenous (primary) regressors
for j in range(n_end):
    for i in range(ncs):
        # Regular expression for the relevant exogenous regressors
        ex_pat = ''.join(['^Z2',str(i+1)])
        # Regular expression for the relevant error term. 
        err_pat = ''.join(['V',str(i+1),',',str(j+1)])
        # Extracting exog regresors converting to numpy array
        pe1 = pd.concat([Ex_df.filter(regex = ex_pat),Inst_df], axis = 1).values
        # Extracting error variable and converting to numpy array
        pe2 = Err_df.filter(regex = err_pat).values
        # Calculating the endogenous primary regressor
        pe = pe1.dot(coeff[j,i,:]).reshape(pe1.shape[0],1) + pe2
        # Constructing the appropriate name for the endo regressor
        end_nm = ''.join(['Z1',str(i+1),',',str(j+1)])
        if j == 0 and i == 0:
            # Initializing the endog df with first calculated regressor
            End_df = pd.DataFrame(pe,columns = [end_nm])
        else:
            # Adding calculated endog regressor onto df
            End_df[end_nm] = pe

# Common Primary Coeff Vector
p_commoncf = np.array(([1,-1]*10)[:n_end+n_exo]).reshape(n_end+n_exo,1)
# Fixed Effect for each crossection
fe = [ 1+x/2 for x in np.arange(0,ncs)]

## Generation of primary regressand
for i in range(ncs):
    # Regular expression for the relevant endogenous regressors
    en_pat = ''.join(['^Z1',str(i+1)])
    # Regular expression for the relevant exogenous regressors
    ex_pat = ''.join(['^Z2',str(i+1)])
    # Name of apporpriate primary error term
    er_nm = ''.join(['e',str(i+1)])
    # Extracting appropriate regressor for primary equation
    pr3 = pd.concat([End_df.filter(regex = en_pat),Ex_df.filter(regex = ex_pat)], axis = 1).values
    # Extracting appropriate error term
    pr4 = Err_df[er_nm].values.reshape(Err_df.shape[0],1)
    # Generating primary regressand
    pr = fe[i] + pr3.dot(p_commoncf)+pr4
    # Constructing the appropriate name for the endo regressor
    pr_nm = ''.join(['Y',str(i+1)])
    if i == 0:
        # Initializing the regressand df 
        Pr_df = pd.DataFrame(pr,columns = [pr_nm])
    else:
        # Adding generated regressand to df
        Pr_df[pr_nm] = pr

Full_df = pd.concat([Pr_df,End_df],axis = 1) 
Full_df = pd.concat([Full_df,Ex_df],axis = 1)
Full_df = pd.concat([Full_df,Inst_df],axis = 1)


In [93]:
cl_nms = list(Full_df.columns)
AX = Full_df.values

In [97]:
Ex = pd.DataFrame(AX,columns = cl_nms)

In [98]:
Ex

,Y1,Y2,Y3,"Z11,1","Z12,1","Z13,1","Z11,2","Z12,2","Z13,2","Z21,1",...,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10
0,1.920257,2.499930,1.802103,-0.238394,5.789692,1.507443,-1.632225,3.716742,3.446418,-0.852707,...,-0.495595,-0.049841,-0.701849,0.184287,-1.802076,-1.769138,-2.382059,-0.352568,-1.213002,-2.723248
1,3.222093,1.494895,-0.278667,1.240612,1.248215,-0.780272,0.418985,1.425320,0.275529,1.151563,...,-0.477877,-1.404646,-1.246429,-0.953009,-0.871884,0.447853,-0.886475,-0.453572,-1.044507,0.090438
2,3.299255,3.496260,3.716134,0.514896,0.892519,1.043001,-2.880998,-0.597051,-0.481151,0.696024,...,0.971406,-0.205509,-0.100712,0.097444,0.677054,0.198065,0.979146,1.086411,0.794844,-0.037421
3,2.113215,8.660916,7.438605,0.782803,3.404751,0.175513,2.025330,-3.206790,-4.787242,-0.189761,...,-0.349566,0.962297,0.811301,0.779851,1.610982,1.066849,0.322317,-0.403073,0.522913,0.413055
4,-3.319633,10.918689,-2.711039,-1.515640,6.030471,1.366682,2.197125,-2.906537,3.796699,-1.147285,...,2.301278,1.144985,1.455181,-0.490109,-1.570736,-0.998790,0.380846,0.119653,-0.070998,-1.266041
5,2.895574,2.680815,1.938463,1.346527,0.562672,1.341125,-1.902321,-0.075338,1.023611,-0.473626,...,-0.172426,1.157842,0.513168,1.020896,-0.300601,1.363629,0.207736,0.176865,0.094232,-0.389955
6,1.363783,3.940354,12.417447,1.686320,1.998601,3.955239,-0.227599,-1.115884,-7.839861,-0.600707,...,0.299059,1.104023,2.075060,1.332432,1.074758,0.481679,-0.164177,-1.023795,0.456906,0.528431
7,7.671668,0.427424,3.267821,4.544729,0.250117,0.415472,-4.169562,4.576610,-1.570156,0.733188,...,-1.048200,-0.623370,-0.873039,0.646068,0.320315,-0.606045,-0.598672,-0.290371,-1.278967,-0.095854
8,5.275368,-0.756222,4.121929,2.552354,-3.136511,-0.242385,-2.475245,0.109127,-1.866670,0.972483,...,-1.350070,-2.446744,0.104432,0.206309,0.298389,0.413102,1.703404,1.401167,-0.276968,0.150398
9,1.220367,6.994402,1.383271,-0.398818,4.411744,-4.865825,-4.518651,-3.253706,-3.339904,-0.255865,...,2.374515,1.304772,0.824633,0.010950,-1.243415,-0.898164,0.019373,0.309510,1.277595,0.875667
